# Обучение модели YOLOv5






## ЗАДАНИЕ

Обучите модель **yolov5**,  используя готовый `train.py` файл репозитория https://github.com/ultralytics/yolov5.git.

Для данной модели изображения и классы объектов должны находиться в папках images и labels, соответсвенно.

Протестируйте модель на валидациооной выборке и выведите на экран полученные изображения с bbox и классами объектов.

Сделайте сравнение текущей модели с Faster RCNN по скорости и качетсву.

### Импорт репозитория

In [17]:
!chcp 65001
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


### Установка необходимых пакетов

In [ ]:
!pip install -r yolov5/requirements.txt

### Импрот необходимых модулей

In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import tarfile

### Загрузка данных

In [2]:
# Загрузка датасета
data_dir = "data/"
if not os.path.exists(f"{data_dir}VOCdevkit"):
    if not os.path.exists(f"{data_dir}VOC.tar"):
        urllib.request.urlretrieve(
            "https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar", f"{data_dir}VOC.tar"
        )
    with tarfile.open(f"{data_dir}VOC.tar") as tar:
        tar.extractall()

### Конвертация аннотаций к формату YOLO v5


### Формат PASCAL VOC 
хранит свои аннотации в XML-файлах. Давайте посмотрим на один такой файл.

In [3]:
!chcp 65001
!type data\VOCdevkit\VOC2012\Annotations\2007_000027.xml 

Active code page: 65001
<annotation>
	<folder>VOC2012</folder>
	<filename>2007_000027.jpg</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>flickr</image>
	</source>
	<size>
		<width>486</width>
		<height>500</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>174</xmin>
			<ymin>101</ymin>
			<xmax>349</xmax>
			<ymax>351</ymax>
		</bndbox>
		<part>
			<name>head</name>
			<bndbox>
				<xmin>169</xmin>
				<ymin>104</ymin>
				<xmax>209</xmax>
				<ymax>146</ymax>
			</bndbox>
		</part>
		<part>
			<name>hand</name>
			<bndbox>
				<xmin>278</xmin>
				<ymin>210</ymin>
				<xmax>297</xmax>
				<ymax>233</ymax>
			</bndbox>
		</part>
		<part>
			<name>foot</name>
			<bndbox>
				<xmin>273</xmin>
				<ymin>333</ymin>
				<xmax>297</xmax>
				<ymax>354</ymax>
			</bndbox>
		</part>

Данная аннотация относится к файлу 2007_000027.jpg размером 486х500х3. Файл содерит один тэг object, описывающий класс объекта и координаты ограничивающих рамок. Класс определяется тегом name. Рамка тегами - xmin, ymin, xmax, ymax. Соответственно верхний левый угол рамки (xmin, ymin) и нижний правый (xmax, ymax). Четыре тега part описывают рамки частей объекта.

### Формат YOLO v5 
подразумевает аннотацию для каждого изображения в виде файла .txt (имя файл совпадает с именем изображения), где каждая строка текстового файла описывает ограничивающую рамку. Рассмотрим следующее изображение.

![picture](https://drive.google.com/uc?export=view&id=1yUJt5XTvYo4MbWxKAxhPdCk-yCnQ1AEz)




Файл аннотаций zidane.txt для приведенного выше изображения выглядит следующим образом:

```
27 0.786719 0.502778 0.0328125 0.161111 0.287125
0 0.471875 0.629167 0.765625 0.713889 0.593856
0 0.741406 0.522222 0.310937 0.925 0.877349
```
Описано три объекта. Спецификация для каждой строки выглядит следующим образом:

* Одна строка на объект
* Каждая строка имеет формат class x_center y_center width height.
* Координаты нормализованы по относительно размера изображения
* Индексация классов начинается с нуля

Структура каталогов набора данных формата YOLOv5
```
base_dir
├── images           # изображения
│   ├── test
│   ├── train
│   └── validation
└── labels           # аннотации
    ├── test
    ├── train
    └── validation
```    


### Подготовка к преобразованию данных

In [4]:

def extract_info_from_xml(xml_file):
    
    '''
        Извлекает описание объектов из формата XML и преобразует данные в словарь
    '''
    root = ET.parse(xml_file).getroot()
    
    # инициализация словаря
    info_dict = {}
    info_dict['bboxes'] = []

    # парсинг XML
    for elem in root:
        # получение имени файла
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # получение размера файла
        elif elem.tag == "size":
            image_size = [-1,-1,-1]
            for subelem in elem:
                if subelem.tag == "width":
                    image_size[0] = int(subelem.text)
                if subelem.tag == "height":
                    image_size[1] = int(subelem.text)
                if subelem.tag == "depth":
                    image_size[2] = int(subelem.text)
            
            info_dict['image_size'] = tuple(image_size)
        
        # получение описания рамок
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [5]:
extract_info_from_xml("data/VOCdevkit/VOC2012/Annotations/2011_003300.xml")


{'bboxes': [{'class': 'person',
   'xmax': 264,
   'xmin': 71,
   'ymax': 302,
   'ymin': 8}],
 'filename': '2011_003300.jpg',
 'image_size': (500, 375, 3)}

In [6]:
# получение словаря объектов для отображения имени на идентификатор
annotations_dir = 'data/VOCdevkit/VOC2012/Annotations'
images_dir = 'data/VOCdevkit/VOC2012/JPEGImages'
a = [os.path.join(annotations_dir, x) for x in os.listdir(annotations_dir) if x[-3:] == "xml"]
a = [extract_info_from_xml(x) for x in a] 
a = [x["bboxes"] for x in  a]
a = [[z["class"] for z in x] for x in a]
a = set(sum(a, []))
a.sort()
print(a)
class_name_to_id_mapping = {x: i for (i, x) in enumerate(a)}

class_name_to_id_mapping


{'tvmonitor': 0,
 'aeroplane': 1,
 'bicycle': 2,
 'car': 3,
 'bottle': 4,
 'horse': 5,
 'boat': 6,
 'motorbike': 7,
 'pottedplant': 8,
 'bus': 9,
 'sheep': 10,
 'cat': 11,
 'person': 12,
 'sofa': 13,
 'cow': 14,
 'bird': 15,
 'chair': 16,
 'train': 17,
 'diningtable': 18,
 'dog': 19}

In [7]:

def convert_to_yolov5(info_dict):
    '''
        Преобразование словаря с описанием в формат yolo и запись на диск
    '''
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join(annotations_dir, info_dict["filename"].replace("jpg", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [8]:
# Получение описаний
annotations = [os.path.join(annotations_dir, x) for x in os.listdir(annotations_dir) if x[-3:] == "xml"]
annotations.sort()

# Конвертация
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join(annotations_dir, x) for x in os.listdir(annotations_dir) if x[-3:] == "txt"]
# annotations

100%|██████████| 17125/17125 [00:18<00:00, 902.43it/s] 


#### Тестирование аннотаций

In [10]:

# Развернем словарь
class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    '''
    Отрисовка изображения и рамок
    '''
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)), width=2)
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))].upper())
    
    plt.figure(figsize=(1, 1))
    plt.imshow(np.array(image), aspect=1)
    plt.show()

# Получим случайное описание
annotation_file = random.choice(annotations)
print(annotation_file)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
print(annotation_list)
# Получим имя файла изображеня 
image_file = annotation_file.replace("Annotations", "JPEGImages").replace("txt", "jpg")
assert os.path.exists(image_file)

image = Image.open(image_file)

plot_bounding_box(image, annotation_list)

data/VOCdevkit/VOC2012/Annotations\2010_006174.txt
[[12.0, 0.466, 0.686, 0.128, 0.399], [12.0, 0.979, 0.614, 0.042, 0.291]]


AssertionError: 

#### Разбиение набора данных

In [ ]:
# Read images and annotations
images_dir = 'data/VOCdevkit/VOC2012/JPEGImages'

images = [os.path.join(images_dir, x) for x in os.listdir(images_dir)]
annotations = [os.path.join(annotations_dir, x) for x in os.listdir(annotations_dir) if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
!mkdir data\VOCdevkit\VOC2012\JPEGImages\train 
!mkdir data\VOCdevkit\VOC2012\JPEGImages\val 
!mkdir data\VOCdevkit\VOC2012\JPEGImages\test 
!mkdir data\VOCdevkit\VOC2012\Annotations\train 
!mkdir data\VOCdevkit\VOC2012\Annotations\val 
!mkdir data\VOCdevkit\VOC2012\Annotations\test

�������� ��� 䠩� data\VOCdevkit\VOC2012\JPEGImages\train 㦥 �������.


In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'data\VOCdevkit\VOC2012\JPEGImages/train')
move_files_to_folder(val_images, 'data\VOCdevkit\VOC2012\JPEGImages/val/')
move_files_to_folder(test_images, 'data\VOCdevkit\VOC2012\JPEGImages/test/')
move_files_to_folder(train_annotations, 'data\VOCdevkit\VOC2012\Annotations/train/')
move_files_to_folder(val_annotations, 'data\VOCdevkit\VOC2012\Annotations/val/')
move_files_to_folder(test_annotations, 'data\VOCdevkit\VOC2012\Annotations/test/')

In [23]:
!chcp 65001
# mv annotations labels
!rename annotations labels
!cd yolov5
!dir /B
!cd yolov5
!dir /B

Active code page: 65001


The system cannot find the file specified.


data
homework_01_YOLO.ipynb
README.md
yolov5
data
homework_01_YOLO.ipynb
README.md
yolov5


#### Файл конфигурации данных
Создадим файл `voc_data.yaml`

```
# путь к данным
train: ./data/VOCdevkit/VOC2021/images/train/ 
val:  ./data/VOCdevkit/VOC2021/images/val/
test: ./data/VOCdevkit/VOC2021/images/test/

# число классов
nc: 20

# имена классов
names: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
        'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
```


#### Конфигурационный файл гиперпараметров
Будем использовать `./yolov5/data/hips/hyp.VOC.yaml`

#### Обучение
`python train.py --batch 128 --weights yolov5m6.pt --data VOC.yaml --epochs 50 --img 512 --hyp hyp.scratch-med.yaml --evolve`
C:\Users\brovkin_vn\AppData\Local\Programs\Python\Python37\python.exe  train.py --batch 128 --weights data/models/yolov5m6.pt --data ../voc_data.yaml --epochs 50 --img 512 --hyp data/hyps/hyp.VOC.yaml --workers=8 --name=my_yolo --evolve

In [5]:
!python yolov5/train.py --batch 128 --weights yolov5/data/models/yolov5m6.pt --data voc_data.yaml --epochs 50 --img 512 --hyp yolov5/data/hyps/hyp.VOC.yaml --workers=8 --name=my_yolo --evolve

github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5


train: weights=yolov5/data/models/yolov5m6.pt, cfg=, data=voc_data.yaml, hyp=yolov5/data/hyps/hyp.VOC.yaml, epochs=50, batch_size=128, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=300, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=my_yolo, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v6.1-191-gd29df68 Python-3.7.9 torch-1.11.0+cpu CPU

hyperparameters: lr0=0.00334, lrf=0.15135, momentum=0.74832, weight_decay=0.00025, warmup_epochs=3.3835, warmup_momentum=0.59462, warmup_bias_lr=0.18657, box=0.02, cls=0.21638, cls_pw=0.5, obj=0.51728, obj_pw=0.67198, iou_t=0.2, anchor_t=3.3744, fl_gamma=0.0, hsv_h=0.01041, hsv_s=0.54703, hsv_v=0.27739, degrees=0.0, translate=